In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

import matplotlib.pyplot as plt
%matplotlib inline

import datetime as dt
import pickle
import os
import time

import seaborn as sns
sns.set()
from matplotlib import style
style.use('ggplot')

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

def MAE(y, y_hat):
    return np.mean(np.abs(y_hat-y))

def MAPE(y, y_hat):
    return np.mean(np.abs(y_hat-y) / (y+1e-3)) * 100

def RMSE(y, y_hat):
    return np.sqrt(np.mean((y_hat - y)**2))

def RMSPE(y, y_hat):
    return np.sqrt(np.mean(((y - y_hat) / (y+1e-3)) ** 2))

rmpse_scorer = make_scorer(RMSPE, greater_is_better = False) # Loss function

def train_scores(model, X_train, y_train, y_test, y_hat):
    mae = MAE(y_test, y_hat)
    mape = MAPE(y_test, y_hat)
    rmse = RMSE(y_test, y_hat)
    rmspe = RMSPE(y_test, y_hat)
    return mae, mape, rmse, rmspe

def get_scores(y, y_hat):
    mae = MAE(y, y_hat)
    mape = MAPE(y, y_hat)
    rmse = RMSE(y, y_hat)
    rmspe = RMSPE(y, y_hat)
    return mae, mape, rmse, rmspe

In [3]:
df_all = pd.read_csv('sp500_wti.csv')
df_all['Date'] = pd.to_datetime(df_all['Date'])
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197718 entries, 0 to 1197717
Data columns (total 34 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Date                       1197718 non-null  datetime64[ns]
 1   Adj Close                  1197718 non-null  float64       
 2   volume_obv                 1197718 non-null  float64       
 3   volume_fi                  1197718 non-null  float64       
 4   volatility_bbm             1197718 non-null  float64       
 5   volatility_bbw             1197718 non-null  float64       
 6   volatility_bbhi            1197718 non-null  float64       
 7   volatility_bbli            1197718 non-null  float64       
 8   trend_macd                 1197718 non-null  float64       
 9   trend_macd_signal          1197718 non-null  float64       
 10  trend_macd_diff            1197718 non-null  float64       
 11  trend_psar_up_indicator    1197718 no

In [4]:
def cat_cont_split(df, maxcard=55, omit_vars=['Date', 'Adj Close', 'Return', 'target_return']):
    """Helper function that returns column names of categorical & continuous features from df."""
    cat_feats, cont_feats = [], []
    for col in df:
        if col in omit_vars: 
            continue
        if (df[col].dtype==int or df[col].dtype==float) and df[col].unique().shape[0] > maxcard:
            cont_feats.append(col)
        else:
            cat_feats.append(col)
    return cat_feats, cont_feats
    
cat_vars, cont_vars = cat_cont_split(df_all)
print(len(cat_vars), 'Categorical Features:')
print(cat_vars)
print(len(cont_vars), 'Continuous Features:')
print(cont_vars)

19 Categorical Features:
['volatility_bbhi', 'volatility_bbli', 'trend_psar_up_indicator', 'trend_psar_down_indicator', 'ticker', 'exchange', 'sector', 'industry', 'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
11 Continuous Features:
['volume_obv', 'volume_fi', 'volatility_bbm', 'volatility_bbw', 'trend_macd', 'trend_macd_signal', 'trend_macd_diff', 'momentum_ao', 'momentum_roc', 'Dayofyear', 'Elapsed']


In [5]:
tickers = df_all.ticker.unique().tolist()
print(len(tickers), tickers)

485 ['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADS', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AGN', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AMD', 'AME', 'AMG', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ARNC', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BIIB', 'BK', 'BKNG', 'BLK', 'BLL', 'BMY', 'BR', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CELG', 'CERN', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COO', 'COP', 'COST', 'COTY', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTL', 'CTSH', 'CTXS', 'CVS', 'CVX', 'CXO', 'D', 'DAL', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DLR', 'DLTR', 'DOV', '

In [8]:
rf_scores_train = pd.DataFrame(columns=['ticker', 'MAE', 'MAPE', 'RMSE', 'RMSPE'])
rf_scores_test = pd.DataFrame(columns=['ticker', 'MAE', 'MAPE', 'RMSE', 'RMSPE'])

tic = time.time()
for i, ticker in enumerate(tickers):
    print("\n##### {} {}:".format(i, ticker))
    df = df_all.loc[df_all.ticker==ticker].reset_index()
    
    cat_features = []
    for v in cat_vars:
        n = len(df[v].unique())
        if n > 1 and n <= 13:
            cat_features.append(v)
    ohe = OneHotEncoder()
    cat_cols = ohe.fit_transform(df[cat_features])
    cat_tsfm = Pipeline(steps=[
        ('ohe', OneHotEncoder())
    ])
    cont_tsfm = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median'))
    #     ('scaler', StandardScaler())
    ])
    preproc = ColumnTransformer(transformers=[
        ('cont', cont_tsfm, cont_vars),
        ('cat', cat_tsfm, cat_features)
    ])

    tsfm_np = preproc.fit_transform(df)
    df_tsfm = pd.DataFrame(tsfm_np, columns=cont_vars + ohe.get_feature_names(cat_features).tolist())
    df_tsfm['Date'] = df['Date']
    df_tsfm['Return'] = df['Return']
    df_tsfm['target_return'] = df['target_return']

    df = df_tsfm
    last_day = df['Date'].max()
    cut_day = last_day + dt.timedelta(-90)
    train_df = df[df['Date'] <= cut_day].sort_values(by='Date', ascending=False).reset_index(drop=True)
    test_df = df[(df['Date'] > cut_day) & (df['Date'] < last_day)].sort_values(by='Date', ascending=False).reset_index(drop=True)
    
    cols = cont_vars + ohe.get_feature_names(cat_features).tolist()
    Xtrain, ytrain = train_df[cols + ['Return']], train_df['target_return']
    Xtest, ytest = test_df[cols + ['Return']], test_df['target_return']

    rf = RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=0)
    rf.fit(Xtrain, ytrain)
    elapsed = time.time() - tic
    
    yhat = rf.predict(Xtrain)
    mae, mape, rmse, rmspe = get_scores(ytrain, y_hat)
    
    yhat = rf.predict(Xtest)
    mae, mape, rmse, rmspe = get_scores(ytest, y_hat)
    print("Elapsed: {} sec".format(elapsed))


##### 0 A:
MAE 1.9614269548536694
MAPE 271.83252724769
RMSE 2.305833687604138
RMSPE 10.812723963695259
Elapsed: 0.5818808078765869 sec

##### 1 AAL:
MAE 2.4500428670273435
MAPE 56.82669841611892
RMSE 3.0312373591147215
RMSPE 1.2502729829407806
Elapsed: 1.1797523498535156 sec

##### 2 AAP:
MAE 2.0439718117913843
MAPE -109.95491217215067
RMSE 2.631732095529964
RMSPE 2.7889202709299323
Elapsed: 1.7763733863830566 sec

##### 3 AAPL:
MAE 1.3844268183302793
MAPE 14.791035489428223
RMSE 1.8853910583076217
RMSPE 2.126065030428682
Elapsed: 2.3708651065826416 sec

##### 4 ABBV:
MAE 1.636013290237379
MAPE 2.0160775776598316
RMSE 2.0656985607043916
RMSPE 0.9954123991009928
Elapsed: 2.863865375518799 sec

##### 5 ABC:
MAE 1.7677342355305323
MAPE -125.73311779024914
RMSE 2.116030830381171
RMSPE 6.830334728980668
Elapsed: 3.459310531616211 sec

##### 6 ABMD:
MAE 2.361026005844637
MAPE 87.63168778970059
RMSE 3.165663971227214
RMSPE 11.666187457266972
Elapsed: 4.054807424545288 sec

##### 7 ABT:
MAE 1

MAE 1.4644032479923137
MAPE -238.95614386930487
RMSE 1.9534007834202611
RMSPE 9.836330283519134
Elapsed: 35.00950241088867 sec

##### 59 BAC:
MAE 1.0139528073410085
MAPE -87.29107048538602
RMSE 1.3386012914755512
RMSPE 5.183279210121471
Elapsed: 35.703598499298096 sec

##### 60 BAX:
MAE 2.4285806963154086
MAPE 849.6918844248088
RMSE 2.6422285248033583
RMSPE 38.417070149794895
Elapsed: 36.3004937171936 sec

##### 61 BBY:
MAE 1.1908238711710109
MAPE -82.303493641963
RMSE 1.414336294929802
RMSPE 2.1200528052585432
Elapsed: 36.898709535598755 sec

##### 62 BDX:
MAE 1.7299081788804231
MAPE -36.97934964872023
RMSE 3.0447796806747442
RMSPE 1.800213246129173
Elapsed: 37.59517574310303 sec

##### 63 BEN:
MAE 2.1326740524274035
MAPE 5836.846256069375
RMSE 2.6744406299590926
RMSPE 256.25465681776257
Elapsed: 38.191612005233765 sec

##### 64 BIIB:
MAE 2.50575082635646
MAPE 182.2084970937917
RMSE 3.900018444891965
RMSPE 5.017657066366976
Elapsed: 38.789042949676514 sec

##### 65 BK:
MAE 1.943417647

MAE 0.9462092716442028
MAPE 41.81828853477237
RMSE 1.0905633276526343
RMSPE 1.3898847619639831
Elapsed: 70.25136804580688 sec

##### 117 CTL:
MAE 2.8359254769417426
MAPE 35.724152414982306
RMSE 3.694333376918086
RMSPE 4.578908823087143
Elapsed: 70.95960426330566 sec

##### 118 CTSH:
MAE 1.3718160258267462
MAPE -5.445528932199832
RMSE 2.5408028689252897
RMSPE 1.0236781054226036
Elapsed: 71.55932760238647 sec

##### 119 CTXS:
MAE 1.2009578803060454
MAPE -27.818404275696412
RMSE 2.053089141889459
RMSPE 2.2204691120838262
Elapsed: 72.15807390213013 sec

##### 120 CVS:
MAE 1.502456123346507
MAPE -105.07686152178495
RMSE 1.9016741039281846
RMSPE 2.4692138049577603
Elapsed: 72.75950336456299 sec

##### 121 CVX:
MAE 1.0196023017808375
MAPE -38.22538031900121
RMSE 1.3527468401782992
RMSPE 1.2419742665871467
Elapsed: 73.4570963382721 sec

##### 122 CXO:
MAE 2.188516473548286
MAPE -107.79207538030195
RMSE 2.665108324663806
RMSPE 2.431383191958049
Elapsed: 74.05251812934875 sec

##### 123 D:
MAE 1

MAE 2.7227649035675827
MAPE -293.7004214575859
RMSE 3.574806247489979
RMSPE 39.163402708211706
Elapsed: 105.18490242958069 sec

##### 175 FFIV:
MAE 1.427046082871337
MAPE -361.1647717219898
RMSE 1.7939189348551712
RMSPE 15.11624843251378
Elapsed: 105.78186058998108 sec

##### 176 FIS:
MAE 1.1357613973933294
MAPE 1.401814776312914
RMSE 1.4811572994972988
RMSPE 1.5285275013328823
Elapsed: 106.3787350654602 sec

##### 177 FISV:
MAE 1.2500122450576387
MAPE 1930.9567851262461
RMSE 1.6012644958054034
RMSPE 84.55681532957102
Elapsed: 106.9760811328888 sec

##### 178 FITB:
MAE 1.1752958648157281
MAPE 44.60122875584601
RMSE 1.3951790805087962
RMSPE 1.3713835999049593
Elapsed: 107.66938543319702 sec

##### 179 FLIR:
MAE 1.7820598263226792
MAPE 23.452060680986374
RMSE 2.1467451921941123
RMSPE 1.171170500104728
Elapsed: 108.2646381855011 sec

##### 180 FLS:
MAE 1.704034125886955
MAPE -23.92876170103205
RMSE 2.1436668045030918
RMSPE 1.0101972101849197
Elapsed: 108.86101913452148 sec

##### 181 FLT:

MAE 1.1177254298671127
MAPE -54.76643993845255
RMSE 1.3793929521309984
RMSPE 1.6862224592652
Elapsed: 139.45833492279053 sec

##### 233 INFO:
MAE 0.7178845876528737
MAPE -45.35166476157723
RMSE 0.8603229702296542
RMSPE 1.582764538915703
Elapsed: 139.85215282440186 sec

##### 234 INTC:
MAE 3.5589212329988
MAPE -193.5015400858566
RMSE 4.250011314848777
RMSPE 26.1628105681698
Elapsed: 140.44499397277832 sec

##### 235 INTU:
MAE 2.067730414888029
MAPE -31.88465992170065
RMSE 2.4207815144517437
RMSPE 4.545187898765111
Elapsed: 141.03915667533875 sec

##### 236 IP:
MAE 1.5063118731139613
MAPE 50.59267363940727
RMSE 2.0214180897062373
RMSPE 1.646283830395375
Elapsed: 141.63301348686218 sec

##### 237 IPG:
MAE 1.2357363704645619
MAPE -25.42938511704435
RMSE 1.9632714020673252
RMSPE 0.8951339675129238
Elapsed: 142.32730746269226 sec

##### 238 IPGP:
MAE 3.173556782718695
MAPE 31.55904559542584
RMSE 3.89358170788199
RMSPE 1.32320261627682
Elapsed: 142.9244306087494 sec

##### 239 IQV:
MAE 1.0459

MAE 1.9975772413656794
MAPE -95.05877717680839
RMSE 2.5990090366461573
RMSPE 10.466429352193337
Elapsed: 173.75856566429138 sec

##### 291 MCK:
MAE 1.6144641554760724
MAPE 53.94572864875919
RMSE 1.9715702489657383
RMSPE 2.702799020818889
Elapsed: 174.35424041748047 sec

##### 292 MCO:
MAE 0.9462429193167933
MAPE -25.864344084524156
RMSE 1.1816462215958425
RMSPE 1.4447297825030105
Elapsed: 174.94885849952698 sec

##### 293 MDLZ:
MAE 1.2562646485621745
MAPE -81.29139269203564
RMSE 2.0111125122752855
RMSPE 6.112016022468141
Elapsed: 175.5390911102295 sec

##### 294 MDT:
MAE 1.1828067344586735
MAPE -31.69972587524431
RMSE 1.6115785246579042
RMSPE 1.6485979231993986
Elapsed: 176.1360502243042 sec

##### 295 MET:
MAE 0.992777654009196
MAPE -85.06497196982711
RMSE 1.3153795404936235
RMSPE 3.618800910710987
Elapsed: 176.8345274925232 sec

##### 296 MGM:
MAE 2.3386282912544543
MAPE 196.1343037430422
RMSE 2.7433468296833734
RMSPE 3.9275711990233146
Elapsed: 177.42911219596863 sec

##### 297 MHK:

MAE 1.1982513455617128
MAPE -32.09539956477726
RMSE 1.5664211296501223
RMSPE 1.56173842078318
Elapsed: 208.70175099372864 sec

##### 349 PEG:
MAE 0.964294629369367
MAPE 0.5383572617251905
RMSE 1.167160013546294
RMSPE 1.5008807450811883
Elapsed: 209.3013129234314 sec

##### 350 PEP:
MAE 0.5703448887003877
MAPE 308.03039944506213
RMSE 0.691037557526064
RMSPE 8.02965235335318
Elapsed: 209.8958866596222 sec

##### 351 PFE:
MAE 1.3168994006486574
MAPE -17.847309232611337
RMSE 1.6580591531379796
RMSPE 1.3442892426026105
Elapsed: 210.5912778377533 sec

##### 352 PFG:
MAE 1.116230107957573
MAPE -209.32660881864442
RMSE 1.388381316983955
RMSPE 8.584949186289474
Elapsed: 211.28771376609802 sec

##### 353 PG:
MAE 0.612712929349898
MAPE 17.368666408507796
RMSE 0.7811617969791941
RMSPE 0.9585832553291984
Elapsed: 211.88112902641296 sec

##### 354 PGR:
MAE 1.1438372974393964
MAPE 160.06788718677896
RMSE 1.710861496124823
RMSPE 4.062057199128419
Elapsed: 212.57939958572388 sec

##### 355 PH:
MAE 1.56

MAE 1.6349072846812387
MAPE -579.1461210896864
RMSE 2.1035091046431122
RMSPE 21.216967600344475
Elapsed: 244.1342899799347 sec

##### 407 STX:
MAE 2.1177351136084512
MAPE 14.41422370280251
RMSE 2.7496857808664665
RMSPE 1.127426417077688
Elapsed: 244.73003482818604 sec

##### 408 STZ:
MAE 1.1624612543740116
MAPE -2190.8581636979216
RMSE 1.3743571443123892
RMSPE 91.04274387429523
Elapsed: 245.32722997665405 sec

##### 409 SWK:
MAE 1.5596010274836487
MAPE -69.15484880378116
RMSE 2.0366851784505635
RMSPE 1.3095168999419389
Elapsed: 246.0241038799286 sec

##### 410 SWKS:
MAE 2.4486624582621044
MAPE -32.93429365744836
RMSE 2.699143383879145
RMSPE 1.2329189188269873
Elapsed: 246.61945629119873 sec

##### 411 SYF:
MAE 1.4223349512621255
MAPE 3588.3858075287435
RMSE 2.5286370820693143
RMSPE 151.59190850637174
Elapsed: 247.01303052902222 sec

##### 412 SYK:
MAE 1.099479054955621
MAPE 49.74930063763671
RMSE 1.2358812672535353
RMSPE 1.7281619716561998
Elapsed: 247.60945916175842 sec

##### 413 SYY

MAE 0.9309254835977858
MAPE 17.447185968487503
RMSE 1.1209649362983103
RMSPE 2.3645054737030957
Elapsed: 278.57263350486755 sec

##### 465 WHR:
MAE 1.939021137707961
MAPE 70.5221297377353
RMSE 2.4245768178108253
RMSPE 3.168272909236133
Elapsed: 279.17109990119934 sec

##### 466 WLTW:
MAE 1.237690547379614
MAPE 58.33192636210134
RMSE 1.6316261164593338
RMSPE 2.5364830810366854
Elapsed: 279.7658772468567 sec

##### 467 WM:
MAE 1.213316734369118
MAPE 100.89267842362734
RMSE 1.4343005166298215
RMSPE 9.390309366806633
Elapsed: 280.4582579135895 sec

##### 468 WMB:
MAE 1.1368735442888815
MAPE 25.202324579483005
RMSE 1.4193771921227774
RMSPE 1.2017728243579944
Elapsed: 281.1556613445282 sec

##### 469 WMT:
MAE 0.8291207355624759
MAPE 74.26880705851752
RMSE 1.0016172509216164
RMSPE 1.881477264245297
Elapsed: 281.7534441947937 sec

##### 470 WRK:
MAE 1.6749472649424888
MAPE 36.61402699368298
RMSE 2.126393998391136
RMSPE 4.735517494165593
Elapsed: 282.1442482471466 sec

##### 471 WU:
MAE 1.42476

In [14]:
rf_preds = pd.DataFrame()
tic = time.time()
for i, ticker in enumerate(tickers):
    try:
        df = df_all.loc[df_all.ticker==ticker].reset_index()

        cat_features = []
        for v in cat_vars:
            n = len(df[v].unique())
            if n > 1 and n <= 13:
                cat_features.append(v)
        ohe = OneHotEncoder()
        cat_cols = ohe.fit_transform(df[cat_features])
        cat_tsfm = Pipeline(steps=[
            ('ohe', OneHotEncoder())
        ])
        cont_tsfm = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median'))
        #     ('scaler', StandardScaler())
        ])
        preproc = ColumnTransformer(transformers=[
            ('cont', cont_tsfm, cont_vars),
            ('cat', cat_tsfm, cat_features)
        ])

        tsfm_np = preproc.fit_transform(df)
        df_tsfm = pd.DataFrame(tsfm_np, columns=cont_vars + ohe.get_feature_names(cat_features).tolist())
        df_tsfm['Date'] = df['Date']
        df_tsfm['Return'] = df['Return']
        df_tsfm['target_return'] = df['target_return']

        df = df_tsfm
        last_day = df['Date'].max()
        cut_day = last_day + dt.timedelta(-90)
        train_df = df[df['Date'] <= cut_day].sort_values(by='Date', ascending=False).reset_index(drop=True)
        test_df = df[(df['Date'] > cut_day) & (df['Date'] < last_day)].sort_values(by='Date', ascending=False).reset_index(drop=True)

        cols = cont_vars + ohe.get_feature_names(cat_features).tolist()
        Xtrain, ytrain = train_df[cols + ['Return']], train_df['target_return']
        Xtest, ytest = test_df[cols + ['Return']], test_df['target_return']

        rf = RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=0)
        rf.fit(Xtrain, ytrain)
        elapsed = time.time() - tic

        yhat = rf.predict(Xtest)
        mae, mape, rmse, rmspe = get_scores(ytest, yhat)


    #     print(yhat)
        df_temp = pd.DataFrame({'target': ytest,
                                'pred': yhat})
    #     df_temp = pd.DataFrame([ytest, yhat], columns=['target', 'pred'])
        df_temp['ticker'] = ticker
    #     print(df_temp.head())
        rf_preds = rf_preds.append(df_temp)
        if i % 50 == 0 or i > len(tickers)-3:
            print("\n##### {} {}:".format(i, ticker))
            print("Elapsed: {} sec".format(elapsed))
    except:
        pass
rf_preds.to_csv('./Results/rf_preds.csv', index=False)


##### 0 A:
Elapsed: 0.47879767417907715 sec

##### 50 ATO:
Elapsed: 30.188323736190796 sec

##### 100 CMG:
Elapsed: 60.283129930496216 sec

##### 150 EIX:
Elapsed: 90.58967113494873 sec

##### 200 GRMN:
Elapsed: 119.75897121429443 sec

##### 250 JNJ:
Elapsed: 149.45165538787842 sec

##### 300 MLM:
Elapsed: 179.05149817466736 sec

##### 350 PEP:
Elapsed: 209.14142084121704 sec

##### 400 SNA:
Elapsed: 239.17720532417297 sec

##### 450 VLO:
Elapsed: 268.99003195762634 sec

##### 483 ZION:
Elapsed: 289.0594582557678 sec

##### 484 ZTS:
Elapsed: 289.5476610660553 sec


In [15]:
rf_preds.head()

,target,pred,ticker
0,-1.200185,-1.266855,A
1,0.409499,-1.182547,A
2,0.046823,-1.034663,A
3,0.399582,-1.727513,A
4,0.781708,-2.533976,A


In [13]:
# for ticker in tickers:
ticker = 'GOOG'
df = df_all.loc[df_all.ticker==ticker].reset_index()
df.head()

,index,Date,Adj Close,volume_obv,volume_fi,volatility_bbm,volatility_bbw,volatility_bbhi,volatility_bbli,trend_macd,trend_macd_signal,trend_macd_diff,trend_psar_up_indicator,trend_psar_down_indicator,momentum_ao,momentum_roc,Return,target_return,ticker,exchange,sector,industry,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,481797,2010-01-05,0.995596,-0.536007,-0.033697,0.997798,0.008807,0.0,0.0,-0.031592,-0.023377,0.006101,0.0,0.0,0.000000,-6.983648,-0.440367,-2.520874,GOOG,NASDAQ,TECHNOLOGY,"COMPUTER SOFTWARE: PROGRAMMING, DATA PROCESSING",2010,1,1,5,1,5,False,False,False,False,False,False,1262649600
1,481798,2010-01-06,0.970499,-2.569901,-0.033697,0.988698,0.051976,0.0,0.0,-0.031592,-0.023377,0.006101,0.0,1.0,0.000000,-6.983648,-2.520874,-2.327951,GOOG,NASDAQ,TECHNOLOGY,"COMPUTER SOFTWARE: PROGRAMMING, DATA PROCESSING",2010,1,1,6,2,6,False,False,False,False,False,False,1262736000
2,481799,2010-01-07,0.947906,-5.848892,-0.033697,0.978500,0.083775,0.0,0.0,-0.031592,-0.023377,0.006101,0.0,0.0,0.000000,-6.983648,-2.327951,1.333112,GOOG,NASDAQ,TECHNOLOGY,"COMPUTER SOFTWARE: PROGRAMMING, DATA PROCESSING",2010,1,1,7,3,7,False,False,False,False,False,False,1262822400
3,481800,2010-01-08,0.960543,-3.433843,-0.033697,0.974909,0.080250,0.0,0.0,-0.031592,-0.023377,0.006101,0.0,0.0,0.000000,-6.983648,1.333112,-0.151160,GOOG,NASDAQ,TECHNOLOGY,"COMPUTER SOFTWARE: PROGRAMMING, DATA PROCESSING",2010,1,1,8,4,8,False,False,False,False,False,False,1262908800
4,481801,2010-01-11,0.959091,-7.121085,-0.033697,0.972272,0.076960,0.0,0.0,-0.031592,-0.023377,0.006101,0.0,0.0,-0.005153,-6.983648,-0.151160,-1.768389,GOOG,NASDAQ,TECHNOLOGY,"COMPUTER SOFTWARE: PROGRAMMING, DATA PROCESSING",2010,1,2,11,0,11,False,False,False,False,False,False,1263168000


In [26]:
df.describe()

,index,Adj Close,volume_obv,volume_fi,volatility_bbm,volatility_bbw,volatility_bbhi,volatility_bbli,trend_macd,trend_macd_signal,trend_macd_diff,trend_psar_up_indicator,trend_psar_down_indicator,momentum_ao,momentum_roc,Return,target_return,Year,Month,Week,Day,Dayofweek,Dayofyear,Elapsed
count,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2.547000e+03
mean,483070.000000,2.095600,13.416843,0.000299,2.081427,0.189790,0.072634,0.051040,0.009876,0.009787,0.000250,0.038869,0.038869,0.021332,0.858527,0.073883,0.074028,2014.570475,6.479780,26.310169,15.708677,2.025520,181.752258,1.422143e+09
std,735.399891,1.076511,18.145148,0.014721,1.064574,0.131087,0.259587,0.220123,0.032088,0.029795,0.010158,0.193321,0.193321,0.081525,5.120582,1.537772,1.537741,2.919051,3.450566,15.051247,8.737865,1.400166,105.408458,9.221774e+07
min,481797.000000,0.695764,-48.888745,-0.091195,0.743367,0.008807,0.000000,0.000000,-0.114357,-0.094396,-0.063626,0.000000,0.000000,-0.350078,-18.339241,-8.377507,-8.377507,2010.000000,1.000000,1.000000,1.000000,0.000000,2.000000,1.262650e+09
25%,482433.500000,1.023191,2.803221,-0.003609,1.009672,0.094494,0.000000,0.000000,-0.009979,-0.009737,-0.004893,0.000000,0.000000,-0.026531,-2.132488,-0.638924,-0.638924,2012.000000,3.000000,13.000000,8.000000,1.000000,90.000000,1.342267e+09
50%,483070.000000,1.815971,20.872396,0.000415,1.808597,0.146529,0.000000,0.000000,0.007546,0.007368,0.000622,0.000000,0.000000,0.015325,0.888349,0.050229,0.050229,2015.000000,6.000000,26.000000,16.000000,2.000000,181.000000,1.422317e+09
75%,483706.500000,3.016274,26.777209,0.004268,3.008127,0.253200,0.000000,0.000000,0.027692,0.026540,0.005510,0.000000,0.000000,0.066065,3.759226,0.846216,0.846216,2017.000000,9.000000,39.000000,23.000000,3.000000,273.000000,1.501934e+09
max,484343.000000,4.870970,34.464171,0.165405,4.738297,0.750989,1.000000,1.000000,0.122520,0.104388,0.051812,1.000000,1.000000,0.348521,29.282815,16.052431,16.052431,2020.000000,12.000000,53.000000,31.000000,4.000000,366.000000,1.581984e+09


In [14]:
ohe = OneHotEncoder()
cat_features = []
for v in cat_vars:
    n = len(df[v].unique())
    print(v, n)
    if n > 1 and n <= 13:
        cat_features.append(v)
print(cat_features)
cat_cols = ohe.fit_transform(df[cat_features])
cat_cols.shape

volatility_bbhi 2
volatility_bbli 2
trend_psar_up_indicator 2
trend_psar_down_indicator 2
ticker 1
exchange 1
sector 1
industry 1
Year 11
Month 12
Week 53
Day 31
Dayofweek 5
Is_month_end 2
Is_month_start 2
Is_quarter_end 2
Is_quarter_start 2
Is_year_end 2
Is_year_start 1
['volatility_bbhi', 'volatility_bbli', 'trend_psar_up_indicator', 'trend_psar_down_indicator', 'Year', 'Month', 'Dayofweek', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end']


(2547, 46)

In [15]:
scaler = StandardScaler()
cont_cols = scaler.fit_transform(df[cont_vars])
cont_cols.shape

(2547, 11)

In [16]:
np.isnan(cont_cols).sum(), (1-np.isfinite(cont_cols)).sum()

(0, 0)

In [18]:
cat_tsfm = Pipeline(steps=[
    ('ohe', OneHotEncoder())
])
cont_tsfm = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
#     ('scaler', StandardScaler())
])
preproc = ColumnTransformer(transformers=[
    ('cont', cont_tsfm, cont_vars),
    ('cat', cat_tsfm, cat_features)
])

tsfm_np = preproc.fit_transform(df)
tsfm_np.shape

df_tsfm = pd.DataFrame(tsfm_np, columns=cont_vars + ohe.get_feature_names(cat_features).tolist())
# tsfm_np[0]
# for v in ['Date', 'Return', 'target_return']:
#     df_tsfm.loc[:, v] = df[:, v]
df_tsfm['Date'] = df['Date']
df_tsfm['Return'] = df['Return']
df_tsfm['target_return'] = df['target_return']
df_tsfm.describe()

,volume_obv,volume_fi,volatility_bbm,volatility_bbw,trend_macd,trend_macd_signal,trend_macd_diff,momentum_ao,momentum_roc,Dayofyear,Elapsed,volatility_bbhi_0.0,volatility_bbhi_1.0,volatility_bbli_0.0,volatility_bbli_1.0,trend_psar_up_indicator_0.0,trend_psar_up_indicator_1.0,trend_psar_down_indicator_0.0,trend_psar_down_indicator_1.0,Year_2010,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017,Year_2018,Year_2019,Year_2020,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Dayofweek_0,Dayofweek_1,Dayofweek_2,Dayofweek_3,Dayofweek_4,Is_month_end_False,Is_month_end_True,Is_month_start_False,Is_month_start_True,Is_quarter_end_False,Is_quarter_end_True,Is_quarter_start_False,Is_quarter_start_True,Is_year_end_False,Is_year_end_True,Return,target_return
count,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2.547000e+03,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.00000,2547.000000,2547.00000,2547.00000,2547.00000,2547.00000,2547.000000,2547.000000,2547.00000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000,2547.000000
mean,13.416843,0.000299,2.081427,0.189790,0.009876,0.009787,0.000250,0.021332,0.858527,181.752258,1.422143e+09,0.927366,0.072634,0.948960,0.051040,0.961131,0.038869,0.961131,0.038869,0.098547,0.09894,0.098155,0.09894,0.09894,0.09894,0.09894,0.098547,0.098547,0.09894,0.012564,0.087161,0.079702,0.085591,0.081272,0.083628,0.083628,0.082843,0.087554,0.079702,0.086376,0.080487,0.082057,0.186887,0.205340,0.204554,0.201806,0.201413,0.966627,0.033373,0.969376,0.030624,0.989792,0.010208,0.991755,0.008245,0.997252,0.002748,0.073883,0.074028
std,18.145148,0.014721,1.064574,0.131087,0.032088,0.029795,0.010158,0.081525,5.120582,105.408458,9.221774e+07,0.259587,0.259587,0.220123,0.220123,0.193321,0.193321,0.193321,0.193321,0.298112,0.29864,0.297582,0.29864,0.29864,0.29864,0.29864,0.298112,0.298112,0.29864,0.111404,0.282127,0.270884,0.279814,0.273306,0.276883,0.276883,0.275698,0.282701,0.270884,0.280974,0.272099,0.274506,0.389897,0.404029,0.403455,0.401427,0.401135,0.179643,0.179643,0.172331,0.172331,0.100538,0.100538,0.090445,0.090445,0.052363,0.052363,1.537772,1.537741
min,-48.888745,-0.091195,0.743367,0.008807,-0.114357,-0.094396,-0.063626,-0.350078,-18.339241,2.000000,1.262650e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-8.377507,-8.377507
25%,2.803221,-0.003609,1.009672,0.094494,-0.009979,-0.009737,-0.004893,-0.026531,-2.132488,90.000000,1.342267e+09,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,-0.638924,-0.638924
50%,20.872396,0.000415,1.808597,0.146529,0.007546,0.007368,0.000622,0.015325,0.888349,181.000000,1.422317e+09,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000

In [24]:
ohe.get_feature_names(cat_features).tolist()

['volatility_bbhi_0.0',
 'volatility_bbhi_1.0',
 'volatility_bbli_0.0',
 'volatility_bbli_1.0',
 'trend_psar_up_indicator_0.0',
 'trend_psar_up_indicator_1.0',
 'trend_psar_down_indicator_0.0',
 'trend_psar_down_indicator_1.0',
 'Year_2010',
 'Year_2011',
 'Year_2012',
 'Year_2013',
 'Year_2014',
 'Year_2015',
 'Year_2016',
 'Year_2017',
 'Year_2018',
 'Year_2019',
 'Year_2020',
 'Month_1',
 'Month_2',
 'Month_3',
 'Month_4',
 'Month_5',
 'Month_6',
 'Month_7',
 'Month_8',
 'Month_9',
 'Month_10',
 'Month_11',
 'Month_12',
 'Dayofweek_0',
 'Dayofweek_1',
 'Dayofweek_2',
 'Dayofweek_3',
 'Dayofweek_4',
 'Is_month_end_False',
 'Is_month_end_True',
 'Is_month_start_False',
 'Is_month_start_True',
 'Is_quarter_end_False',
 'Is_quarter_end_True',
 'Is_quarter_start_False',
 'Is_quarter_start_True',
 'Is_year_end_False',
 'Is_year_end_True']

In [26]:
df = df_tsfm
last_day = df['Date'].max()
cut_day = last_day + dt.timedelta(-30)
train_df = df[df['Date'] <= cut_day].sort_values(by='Date', ascending=False).reset_index(drop=True)
test_df = df[(df['Date'] > cut_day) & (df['Date'] < last_day)].sort_values(by='Date', ascending=False).reset_index(drop=True)
train_df.shape, test_df.shape, train_df.Date.min(), train_df.Date.max(), test_df.Date.min(), test_df.Date.max()

((2527, 60),
 (19, 60),
 Timestamp('2010-01-05 00:00:00'),
 Timestamp('2020-01-17 00:00:00'),
 Timestamp('2020-01-21 00:00:00'),
 Timestamp('2020-02-14 00:00:00'))

In [27]:
train_df.columns

Index(['volume_obv', 'volume_fi', 'volatility_bbm', 'volatility_bbw',
       'trend_macd', 'trend_macd_signal', 'trend_macd_diff', 'momentum_ao',
       'momentum_roc', 'Dayofyear', 'Elapsed', 'volatility_bbhi_0.0',
       'volatility_bbhi_1.0', 'volatility_bbli_0.0', 'volatility_bbli_1.0',
       'trend_psar_up_indicator_0.0', 'trend_psar_up_indicator_1.0',
       'trend_psar_down_indicator_0.0', 'trend_psar_down_indicator_1.0',
       'Year_2010', 'Year_2011', 'Year_2012', 'Year_2013', 'Year_2014',
       'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019',
       'Year_2020', 'Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5',
       'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11',
       'Month_12', 'Dayofweek_0', 'Dayofweek_1', 'Dayofweek_2', 'Dayofweek_3',
       'Dayofweek_4', 'Is_month_end_False', 'Is_month_end_True',
       'Is_month_start_False', 'Is_month_start_True', 'Is_quarter_end_False',
       'Is_quarter_end_True', 'Is_quarter_start_Fa

In [28]:
# df.shape, df_.shape, 
cols = cont_vars + ohe.get_feature_names(cat_features).tolist()
Xtrain, ytrain = train_df[cols + ['Return']], train_df['target_return']
Xtest, ytest = test_df[cols + ['Return']], test_df['target_return']

In [29]:
rf = RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=0)
tic = time.time()
rf.fit(Xtrain, ytrain)
elapsed = time.time() - tic
print("Elapsed: {} sec".format(elapsed))

Elapsed: 0.11347007751464844 sec


In [30]:


yhat = rf.predict(Xtest)
score(rf, Xtrain, ytrain, ytest, yhat)
# print(y, yhat)

RMSE 1.88407193936513
RMSPE 15.369665545172895


In [31]:
for i in range(len(ytest)):
    print(i, ytest[i], yhat[i])

0 -0.07035695095369833 0.3741231964732362
1 0.40140730708460026 0.35395282355135455
2 -0.2377696526390638 0.49700610708436577
3 0.6283167454260496 -0.19288845118430875
4 0.007290170721896415 0.2972490296681119
5 1.990905648954944 0.18492642573864299
6 0.20322036807891572 0.7405640320041739
7 1.9333945835686484 -0.3146399813665179
8 0.08016434745690137 -2.0442969720402124
9 -2.615852366173166 -1.94713080554023
10 3.605416261107552 -2.179866210399415
11 -1.484365442556479 -2.5335339769756464
12 -0.19127805222436398 -1.3975615477526637
13 0.4178791956415795 -1.3422168840264137
14 1.301348341026265 -1.2893355760885858
15 -2.2369750937305866 -2.1073456497824674
16 -1.3412748897926765 -2.23040754846578
17 0.047112841292906715 -2.0327906671750564
18 0.10441435814609877 -1.0846436357837852
